# Satellite-Derived SOC Calculation

Calculate Soil Organic Carbon (SOC) from Satellite Spectral Indices

**Objective**: Create continuous SOC estimates from NDVI, NDWI, BUI, LST (1988-2025)

**Data Sources**:
- Field measured SOC: 1985, 2025
- Satellite indices: 1988-2025

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)

print("Libraries loaded successfully!")

## 1. Load Data

In [ ]:
# Load satellite indices
indices = pd.read_csv('geodata/indices_1985_2025.csv')
indices.drop(columns=['system:index', '.geo'], inplace=True)
indices = indices[3:]  # Start from 1988
indices.reset_index(drop=True, inplace=True)

# Load field-measured SOC
topsoil_2025 = pd.read_csv('data/PresentTopSoil.csv')
topsoil_1985 = pd.read_csv('data/PreviousTopSoil.csv')

# Calculate mean SOC
soc_2025_mean = topsoil_2025['SOC%'].mean()
soc_1985_mean = topsoil_1985['SOC%'].mean()

print(f"Satellite data loaded: {len(indices)} years (1988-2025)")
print(f"Field SOC 1985: {soc_1985_mean:.3f}%")
print(f"Field SOC 2025: {soc_2025_mean:.3f}%")
print(f"\nIndices data shape: {indices.shape}")
print(f"Columns: {list(indices.columns)}")

## 2. Prepare Data for Modeling

In [ ]:
# Replace sentinel values
indices.replace(-9999, np.nan, inplace=True)

# Get complete data
indices_clean = indices.dropna()

print(f"Clean indices data: {len(indices_clean)} years with complete spectral data")
print(f"Year range: {int(indices_clean['year'].min())}-{int(indices_clean['year'].max())}")

# Display statistics
print("\nSpectral Indices Statistics:")
print(indices_clean[['mean_ndvi', 'mean_ndwi', 'mean_bui', 'mean_lst']].describe())

## 3. Create Training Data with Interpolation

In [ ]:
# Interpolate SOC for all years between 1985-2025
def interpolate_soc(year):
    if year < 1985:
        return soc_1985_mean
    elif year > 2025:
        return soc_2025_mean
    else:
        slope = (soc_2025_mean - soc_1985_mean) / (2025 - 1985)
        return soc_1985_mean + slope * (year - 1985)

# Create training features and targets
X = indices_clean[['mean_ndvi', 'mean_ndwi', 'mean_bui', 'mean_lst']].values
y = indices_clean['year'].apply(interpolate_soc).values

print(f"Training data prepared:")
print(f"  Features (X) shape: {X.shape}")
print(f"  Target (y) shape: {y.shape}")
print(f"  Y values range: {y.min():.3f} - {y.max():.3f}%")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"\nFeatures standardized (mean=0, std=1)")

## 4. Build Regression Models

In [ ]:
# Linear Regression
model_lr = LinearRegression()
model_lr.fit(X_scaled, y)
r2_lr = model_lr.score(X_scaled, y)

# Ridge Regression
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X_scaled, y)
r2_ridge = model_ridge.score(X_scaled, y)

print("Model Performance:")
print(f"  Linear Regression R²: {r2_lr:.4f}")
print(f"  Ridge Regression R²: {r2_ridge:.4f}")

# Use the better model
model = model_ridge if r2_ridge > r2_lr else model_lr
model_name = 'Ridge' if r2_ridge > r2_lr else 'Linear'
print(f"\nSelected Model: {model_name} Regression")

# Display feature importance
print(f"\nModel Coefficients:")
features = ['NDVI', 'NDWI', 'BUI', 'LST']
for feat, coef in zip(features, model.coef_):
    print(f"  {feat}: {coef:.6f}")

## 5. Predict SOC for All Years

In [ ]:
# Predict for all years with complete indices
X_all = indices_clean[['mean_ndvi', 'mean_ndwi', 'mean_bui', 'mean_lst']].values
X_all_scaled = scaler.transform(X_all)
soc_predicted = model.predict(X_all_scaled)

# Create results dataframe
results = pd.DataFrame({
    'year': indices_clean['year'].astype(int),
    'mean_ndvi': indices_clean['mean_ndvi'].values,
    'mean_ndwi': indices_clean['mean_ndwi'].values,
    'mean_bui': indices_clean['mean_bui'].values,
    'mean_lst': indices_clean['mean_lst'].values,
    'soc_predicted': np.round(soc_predicted, 4)
})

print(f"Predicted SOC values for {len(results)} years\n")
print("First 10 years:")
print(results.head(10).to_string(index=False))
print("\n...\n")
print("Last 10 years:")
print(results.tail(10).to_string(index=False))

## 6. Analysis & Comparison

In [ ]:
print("FIELD MEASUREMENTS vs SATELLITE-DERIVED PREDICTIONS")
print("="*60)
print(f"1985 Field Measured SOC:     {soc_1985_mean:.4f}%")
print(f"2025 Field Measured SOC:     {soc_2025_mean:.4f}%")

soc_1988 = results[results['year'] == 1988]['soc_predicted'].values
soc_2025_pred = results[results['year'] == 2025]['soc_predicted'].values

if len(soc_1988) > 0:
    print(f"1988 Predicted SOC:          {soc_1988[0]:.4f}%")
if len(soc_2025_pred) > 0:
    print(f"2025 Predicted SOC:          {soc_2025_pred[0]:.4f}%")

# Trend analysis
soc_change = results['soc_predicted'].iloc[-1] - results['soc_predicted'].iloc[0]
print(f"\nPredicted SOC Change:        {soc_change:+.4f}% over {len(results)} years")

# Identify extremes
max_soc_idx = results['soc_predicted'].idxmax()
min_soc_idx = results['soc_predicted'].idxmin()
print(f"\nHighest SOC Year:            {int(results.loc[max_soc_idx, 'year'])} ({results.loc[max_soc_idx, 'soc_predicted']:.4f}%)")
print(f"Lowest SOC Year:             {int(results.loc[min_soc_idx, 'year'])} ({results.loc[min_soc_idx, 'soc_predicted']:.4f}%)")

print(f"\nModel R² Score:              {model.score(X_all_scaled, y):.4f}")
print(f"\n({model.score(X_all_scaled, y)*100:.1f}% of SOC variation explained by spectral indices)")

## 7. Save Results

In [ ]:
# Save predictions
output_file = 'geodata/soc_satellite_derived.csv'
results.to_csv(output_file, index=False)
print(f"✓ Saved to: {output_file}")
print(f"  File size: {len(results)} rows, {len(results.columns)} columns")

## 8. Visualization - Time Series

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

# Plot satellite-derived SOC
ax.plot(results['year'], results['soc_predicted'], 'b-o', linewidth=2.5, markersize=5, label='Satellite-Derived SOC', alpha=0.8)

# Add field measurements as horizontal lines
ax.axhline(y=soc_1985_mean, color='green', linestyle='--', linewidth=2.5, label=f'Field SOC 1985: {soc_1985_mean:.3f}%')
ax.axhline(y=soc_2025_mean, color='red', linestyle='--', linewidth=2.5, label=f'Field SOC 2025: {soc_2025_mean:.3f}%')

# Add shaded regions for decades
ax.axvspan(1985, 1995, alpha=0.1, color='gray')
ax.axvspan(2015, 2025, alpha=0.1, color='orange')

ax.set_xlabel('Year', fontsize=13, fontweight='bold')
ax.set_ylabel('SOC (%)', fontsize=13, fontweight='bold')
ax.set_title('Satellite-Derived SOC Temporal Trend (1988-2025)\nCompared with Field Measurements', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3, linestyle=':')
ax.set_xlim(1987, 2026)

plt.tight_layout()
plt.savefig('Figure/SOC_Satellite_TimeSeries.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved visualization to: Figure/SOC_Satellite_TimeSeries.png")

## 9. Visualization - Index Relationships

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: NDVI vs SOC
scatter1 = axes[0, 0].scatter(results['mean_ndvi'], results['soc_predicted'], c=results['year'], cmap='viridis', s=100, alpha=0.7, edgecolors='black')
axes[0, 0].set_xlabel('Mean NDVI', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Predicted SOC (%)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('NDVI vs Predicted SOC', fontsize=12, fontweight='bold')
cbar1 = plt.colorbar(scatter1, ax=axes[0, 0])
cbar1.set_label('Year')

# Plot 2: NDWI vs SOC
scatter2 = axes[0, 1].scatter(results['mean_ndwi'], results['soc_predicted'], c=results['year'], cmap='plasma', s=100, alpha=0.7, edgecolors='black')
axes[0, 1].set_xlabel('Mean NDWI', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Predicted SOC (%)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('NDWI vs Predicted SOC', fontsize=12, fontweight='bold')
cbar2 = plt.colorbar(scatter2, ax=axes[0, 1])
cbar2.set_label('Year')

# Plot 3: BUI vs SOC
scatter3 = axes[1, 0].scatter(results['mean_bui'], results['soc_predicted'], c=results['year'], cmap='cool', s=100, alpha=0.7, edgecolors='black')
axes[1, 0].set_xlabel('Mean BUI (Built-Up Index)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Predicted SOC (%)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('BUI vs Predicted SOC', fontsize=12, fontweight='bold')
cbar3 = plt.colorbar(scatter3, ax=axes[1, 0])
cbar3.set_label('Year')

# Plot 4: All indices over time
ax4 = axes[1, 1]
ax4_twin = ax4.twinx()
ax4_twin2 = ax4.twinx()
ax4_twin2.spines['right'].set_position(('outward', 60))

p1, = ax4.plot(results['year'], results['mean_ndvi'], 'g-o', linewidth=2, markersize=4, label='NDVI')
p2, = ax4_twin.plot(results['year'], results['mean_ndwi'], 'b-s', linewidth=2, markersize=4, label='NDWI')
p3, = ax4_twin2.plot(results['year'], results['soc_predicted'], 'r-^', linewidth=2, markersize=4, label='Predicted SOC')

ax4.set_xlabel('Year', fontsize=11, fontweight='bold')
ax4.set_ylabel('NDVI', color='g', fontsize=11, fontweight='bold')
ax4_twin.set_ylabel('NDWI', color='b', fontsize=11, fontweight='bold')
ax4_twin2.set_ylabel('SOC (%)', color='r', fontsize=11, fontweight='bold')
ax4.tick_params(axis='y', labelcolor='g')
ax4_twin.tick_params(axis='y', labelcolor='b')
ax4_twin2.tick_params(axis='y', labelcolor='r')
ax4.set_title('All Indices Over Time', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

lines = [p1, p2, p3]
labels = [l.get_label() for l in lines]
ax4.legend(lines, labels, loc='upper left', fontsize=10)

plt.tight_layout()
plt.savefig('Figure/SOC_Index_Relationships.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved visualization to: Figure/SOC_Index_Relationships.png")

## 10. Correlation Analysis

In [ ]:
# Calculate correlations
corr_matrix = results[['mean_ndvi', 'mean_ndwi', 'mean_bui', 'mean_lst', 'soc_predicted']].corr()

print("\nCORRELATION MATRIX")
print("="*70)
print(corr_matrix.round(4))

# Extract SOC correlations
soc_corr = corr_matrix['soc_predicted'].drop('soc_predicted')
print("\nCORRELATIONS WITH PREDICTED SOC:")
print("="*70)
for index, value in soc_corr.sort_values(ascending=False).items():
    strength = 'Very Strong' if abs(value) > 0.7 else 'Strong' if abs(value) > 0.5 else 'Moderate' if abs(value) > 0.3 else 'Weak'
    direction = 'positive' if value > 0 else 'negative'
    print(f"  {index:12} : {value:+.4f}  ({strength} {direction})")

## 11. Correlation Heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, ax=ax, cbar_kws={'label': 'Correlation Coefficient'},
            linewidths=1, linecolor='gray')

ax.set_title('Spectral Indices & Satellite-Derived SOC Correlation Matrix', 
             fontsize=13, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('Figure/SOC_Correlation_Heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved visualization to: Figure/SOC_Correlation_Heatmap.png")

## 12. Summary & Recommendations

In [ ]:
print("\n" + "="*80)
print("SATELLITE-DERIVED SOC MODEL - SUMMARY")
print("="*80)

print(f"""
MODEL SPECIFICATIONS:
  Algorithm:              {model_name} Regression
  Features:               NDVI, NDWI, BUI, LST
  Training Period:        1988-2025 ({len(results)} years)
  Data Points:            {len(results)}
  
PERFORMANCE METRICS:
  R² Score:               {model.score(X_all_scaled, y):.4f}
  Variance Explained:     {model.score(X_all_scaled, y)*100:.1f}%
  
PREDICTION RESULTS:
  SOC Range:              {results['soc_predicted'].min():.4f}% - {results['soc_predicted'].max():.4f}%
  Mean SOC:               {results['soc_predicted'].mean():.4f}%
  SOC Std Dev:            {results['soc_predicted'].std():.4f}%
  
FIELD vs SATELLITE:
  Field SOC 1985:         {soc_1985_mean:.4f}%
  Field SOC 2025:         {soc_2025_mean:.4f}%
  Field Change:           {soc_2025_mean - soc_1985_mean:+.4f}%
  Predicted Change:       {soc_change:+.4f}%

KEY FINDINGS:
  • Spectral indices explain {model.score(X_all_scaled, y)*100:.1f}% of SOC variation
  • Strongest correlation: {soc_corr.abs().idxmax()} ({soc_corr.abs().max():.4f})
  • Weakest correlation:  {soc_corr.abs().idxmin()} ({soc_corr.abs().min():.4f})
  • Highest SOC year:     {int(results.loc[max_soc_idx, 'year'])} ({results.loc[max_soc_idx, 'soc_predicted']:.4f}%)
  • Lowest SOC year:      {int(results.loc[min_soc_idx, 'year'])} ({results.loc[min_soc_idx, 'soc_predicted']:.4f}%)

OUTPUT FILES CREATED:
  1. geodata/soc_satellite_derived.csv
  2. Figure/SOC_Satellite_TimeSeries.png
  3. Figure/SOC_Index_Relationships.png
  4. Figure/SOC_Correlation_Heatmap.png

RECOMMENDATIONS FOR IMPROVEMENT:
  1. Add climate variables (temperature, rainfall) for better prediction
  2. Develop location-specific models for each of 9 sites
  3. Try non-linear models (Random Forest, XGBoost) for better fit
  4. Include more intermediate soil samples for model validation
  5. Incorporate time-lag effects (vegetation change → SOC change delay)
""")

print("="*80)
print("✓ Satellite-Derived SOC Model Complete!")
print("="*80)